In [1]:
import numpy as np 
from itertools import combinations
from tqdm import tqdm

# Cellular sheaves on graphs 
## Learning sheaf laplacian through minimum total variation approach 

### Generating a toy-case topology

In [2]:
# Let's generate a toy topology for our example

nodes = [i for i in range(7)]
edges = [
    (0,1),
    (0,2),
    (0,6),
    (1,3),
    (1,5),
    (2,3),
    (2,4),
    (3,4),
    (4,6),
    (5,6)
]

V = 7
E = len(edges)

In [3]:
d = 3                                           # Node and edges stalks dimension

F = {
    e:{
        e[0]:np.random.randn(3,3),
        e[1]:np.random.randn(3,3)
        } 
        for e in edges
    }                                           # Incidency linear maps

In [4]:
F

{(0,
  1): {0: array([[-0.23388299,  0.78865359,  0.61799611],
         [ 0.79278694, -1.02686787, -1.64869593],
         [ 3.41236327,  1.29897165, -0.23570743]]), 1: array([[-0.1319878 , -0.91575927,  1.04947683],
         [-1.13442073,  0.51131713,  0.63624677],
         [-1.50033073, -0.54512226,  1.04285424]])},
 (0,
  2): {0: array([[ 2.69097246, -0.76361449, -1.39696753],
         [-1.08966757,  0.26448772, -0.10287909],
         [-0.88414378, -0.38565169,  0.19682273]]), 2: array([[-0.62138699, -0.31974351, -1.0711193 ],
         [ 0.71451497, -0.08846464, -0.10062281],
         [-1.06553994,  0.18660679,  0.3641082 ]])},
 (0,
  6): {0: array([[ 0.69869007, -0.80069847,  1.04142954],
         [ 0.57780399,  0.6107473 , -1.75095531],
         [-2.07549269,  0.09005811, -0.40803112]]), 6: array([[-2.12256690e-01, -1.52018126e-03, -8.61149804e-01],
         [-6.13486735e-01, -2.26432406e+00, -8.88558903e-01],
         [-7.63208014e-01, -4.88185359e-01, -9.02861861e-01]])},
 (1,
  

In [5]:
# Graph representation

A = np.zeros((7,7))

for edge in edges:
    u = edge[0] 
    v = edge[1] 

    A[u,v] = 1
    A[v,u] = 1

D = np.diag(np.sum(A, axis = 0))
L = D - A

In [6]:
# Sheaf representation 

B = np.zeros((d*E, d*V))

for i in range(len(edges)):
    edge = edges[i]

    u = edge[0] 
    v = edge[1] 

    B_u = F[edge][u]
    B_v = F[edge][v]

    B[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
    B[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

L_f = B.T @ B

In [7]:
B

array([[-2.33882994e-01,  7.88653593e-01,  6.17996109e-01,
         1.31987804e-01,  9.15759269e-01, -1.04947683e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 7.92786942e-01, -1.02686787e+00, -1.64869593e+00,
         1.13442073e+00, -5.11317130e-01, -6.36246770e-01,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.41236327e+00,  1.29897165e+00, -2.35707429e-01,
         1.50033073e+00,  5.45122265e-01, -1.04285424e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e

### Generating a smooth signals dataset 

*(from Hansen J., "Learning sheaf Laplacians from smooth signals")* 

In order to retrieve a dataset of smoothsignals, first of all we sample random gaussians vectors on the nodes of the graph. Then we smooth them according to their expansion in terms of the eigenvectors of the sheaf Laplacian $L_0$.

So let's firstly define a dataset of random gaussian vectors. 

In [8]:
N = 100
X = np.random.randn(V*d,N)

Now we'll use the Fourier-domain embedded in the Laplacian spectrum. 

We'll consider a Tikhonov inspired procedure where we firstly project our dataset over the space spanned by the eigenvectors of the sheaf laplacian: namely $U$ the matrix collecting this eigenvectors we have 
\begin{equation}
    \hat{x} = U^T x
\end{equation}

So that defining $h(\lambda) = \frac{1}{1 + 10\lambda}$ and $H = \mathrm{diag}\{h(\lambda)\}_{\lambda}$, we now have

\begin{equation}
    \hat{y} = H(\Lambda) \hat{x}
\end{equation}

and finally our dataset is just reprojected back into the vertex domain:

\begin{equation}
    y = U H(\Lambda) \hat{x} = U H(\Lambda) U^T x
\end{equation}

In [9]:
Lambda, U = np.linalg.eig(L_f)
H = 1/(1 + 10*Lambda)

In [10]:
Y = U @ np.diag(H) @ U.T @ X

Now we deploy the linear map learning strategy. If we consider each of the linear maps connecting the stalks over the nodes with the stalk of the inciding edge, we have that the minimum total variation on the Laplacian can be rewritten as:

\begin{equation}
    \min_{\mathcal{F}_{u \triangleleft e}, \mathcal{F}_{v \triangleleft e}, e \in E} \frac{1}{2} \sum_{e \in E} \sum_{i=1}^N ||\mathcal{F}_{u \triangleleft e}x_u^i - \mathcal{F}_{v \triangleleft e}x_v^i||_F^2
\end{equation}

Clearly this problem can be decomposed into the contribution of each of the edges, requiring us to solve a subproblem for each of the possible edges: 

\begin{equation}
    \min_{\mathcal{F}_{u \triangleleft e}, \mathcal{F}_{v \triangleleft e}} \frac{1}{2} \sum_{i=1}^N ||\mathcal{F}_{u \triangleleft e}x_u^i - \mathcal{F}_{v \triangleleft e}x_v^i||_F^2
\end{equation}

This problem can be solved in a successive convex approximation fashion, being block-wise convex: the update equations are: 

\begin{gather}
    \hat{{\mathcal{F}}}_{u \triangleleft e} = {\mathcal{F}_{v \triangleleft e}} (X_vX_u^T) (X_uX_u^T)^{-1} \\ \nonumber
    \hat{{\mathcal{F}}}_{v \triangleleft e} = {\mathcal{F}_{u \triangleleft e}} (X_uX_v^T) (X_vX_v^T)^{-1}
\end{gather}

In the end we can compute the total energy related to each edge: this means that if we sort out all the edges with respect to this measure we can rebuild the laplacian considering the given $t_0$ number of edges and the associated linear maps. 

In [11]:
# Alternated Linear Maps Learning

def ALML(X_u, X_v, d, T = 500):
    # Initialization 

    F_u = np.random.randn(d)
    F_v = np.random.randn(d)
    gamma = 0.99

    # This matrices can be computed out of the learning loop 

    vu = X_v @ X_u.T
    uv = X_u @ X_v.T
    uu = np.linalg.inv(X_u @ X_u.T)
    vv = np.linalg.inv(X_v @ X_v.T)

    # Alternated learning through blockwise convex programs

    for _ in range(T): 
        # Local step
        F_u_hat = F_v @ vu @ uu
        F_v_hat = F_u @ uv @ vv

        # Convex smoothing
        F_u = F_u + gamma*(F_u_hat - F_u)
        F_v = F_v + gamma*(F_v_hat - F_v)

        gamma *= 0.9
        
    return F_u, F_v 

In [12]:
all_edges = list(combinations(nodes, 2))
maps = {
    e:{
        e[0] : np.zeros((3,3)), 
        e[1] : np.zeros((3,3))
        } 
    for e in all_edges
    }

energies = {
    e : 0
    for e in all_edges
    }

In [13]:
for e in tqdm(all_edges):
    u = e[0]
    v = e[1]

    X_u = Y[u*d:(u+1)*d,:]
    X_v = Y[v*d:(v+1)*d,:]

    F_u, F_v = ALML(X_u, X_v, d, T = 500)

    maps[e][u] = F_u
    maps[e][v] = F_v

    L = 0

    for i in range(100):
        x_u = X_u[:,i]
        x_v = X_v[:,i]
        L += np.linalg.norm(F_u @ x_u - F_v @ x_v)
        
    energies[e] = L

100%|██████████| 21/21 [00:00<00:00, 151.08it/s]


In [14]:
retrieved = sorted(energies.items(), key=lambda x:x[1])[:E]

In [15]:
B_hat = np.zeros((d*E, d*V))

for i in range(10):
    edge = retrieved[i][0]

    u = edge[0] 
    v = edge[1] 

    B_u = maps[edge][u]
    B_v = maps[edge][v]

    B_hat[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
    B_hat[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

L_f_hat = B_hat.T @ B_hat

In [22]:
# The metric chosen by Hansen for the evaluation was the average entry-wise euclidean distance

np.sqrt(np.sum((L_f - L_f_hat)**2)) / L_f.size

0.12498058266325059